## Théorie

DDGP = double deterministic policy gradient

### Politique et $Q$-fonction associée


Dans l'algo précédent, nous approximions la matrice $Q^{opt}$ par une suite de réseau de neurones $Q^n$ puis nous en déduisions la politique.

L'idée des algo de "Gradient de Politique" est de trouver directement une politique à l'aide d'un réseau de neurone.

Une politique Markovienne est une application $\mu$ qui associe à un état $s$ une action $\mu(s)$. Une politique permet ainsi de construire une trajectoire et de calculer une récompense-cumulée. Nous notons
$$
Q^{\mu}[s,a] :=  \sum_{t\geq 0} \gamma^t r_t
$$
où les $r_t$ sont calculée à partir de la trajectoire donnée par
*  $(s_0,a_0)=(s,a)$.
* $s_1,r_1=env(s_0,a_0)$.
* $a_1 = \mu(s_1)$
* $s_2,r_2 = env(s_1,a_1)$
* etc


***Théorème*** Cette Q-fonction satisfait l'équation de Bellman:
$$
Q^{\mu}[s,a] = r + \gamma Q^{\mu}[s',\mu(s')]
$$
où $s',r=env(s,a)$

Démonstration: Notons $(s_0,a_0)=(s,a)$ le point de départ de la trajectoire.
$$
Q^{\mu}[s_0,a_0] = r_0 + \gamma\sum_{t\geq 1} \gamma^{t-1} r_t
$$
$$
 = r_0 + \gamma\sum_{t\geq 0} \gamma^{t} r_{t+1}
$$
Mais par ailleurs $r_1,r_2,...$ est la suite des récompenses décallée d'un temps, c'est donc les récompenses optenues à partir de la trajectoire suivant la politique $\mu$ et démarrant en $(s_1,a_1)=(s_1,\mu(s_1))$ donc
$$
\sum_{t\geq 0} \gamma^{t} r_{t+1} =  Q^{\mu}(s_1,\mu(s_1))
$$
En branchant cela dans l'équation précédente on obtient Bellman.


Ainsi toute polique Markovienne implique une équation de Bellman.










### La $Q$-fonction optimale

Précédemment nous avons définit:
$$
Q^{opt}[s,a] := \max \Big( \sum_{t\geq 0} \gamma^t r_t  \ \big | \  (s_t,a_t)_{t\geq 0} : s_0=s,a_0=a\Big)
$$
C'est un petit miracle: La trajectoire qui donne cette récompense-cumulée est donnée par la politique-Markovienne suivante:
$$
\mu^{opt}(s)= \text{argmax}_a Q^{opt}[s,a]
$$
En d'autre terme:
$$
Q^{opt}= Q^{\mu^{opt}}
$$
Et du coup $Q^{opt}$ est la plus grande de toute les $Q$-fonction.
$$
Q^{opt} = \max_{\mu} Q^{\mu}
$$

* Le but du TP précédent était d'apprendre $Q^{opt}$ (ce qui nous donne accés à $\mu^{opt}$).
* Le but de ce TP est d'approcher simultanément $\mu^{opt}$ et $Q^{opt}$.

### Acteur et critique



L'idée de l'algo est très simple:
 on va en même temps:
* Constuire des paires $(\mu^n,Q^n)$ qui résolvent de plus en plus Bellman,  en diminuant la distance entre:
$$
Q^{n+1}(s,a) \qquad \text{ et } \qquad r + \gamma Q^n(s',\mu^n(s'))
$$
pour tout $s,a,s',r$ tel que $s',r=env(s,a)$

* Tout en maximisant les $Q^n$.


Vocabulaire:
* les $\mu^n$ sont appelés les acteurs; puisqu'ils donnent la politique.
* les $Q^n$ sont appelés les critiques puisque $Q^n(s,a)$ est estime la "valeur" de $(s,a)$.


Attention, au début de l'algo, l'acteur $\mu^0$ et le critique $Q^0$ ne sont aucunement liée: on prend deux réseaux de neurone initialisés aléatoirement.

 Mais au fil du processus d'optimisation, $\mu^n$ et $Q^n$ deviennent intimement liés (puisqu'ils se rapproche de $\mu^{opt}$ et $Q^{opt}$).

### Version "simple"

On choisis des  réseaux de neurones:
* Le critique $Q(s,a)$
* L'acteur $\mu(s)$

On initialise le buffer `R` qui a une capacité limité (first-in first-out). Ainsi les anciens enregistrement (moins bon que les récent) seront oubliés.

`for` episode = 1,M `do`
* On choisit un état initial $s_1$
* `for` $t=1,...,T$ `do`
    * on sélectionne une action $a_t$ selon une politique d'exploration qui est basée sur $\mu$ (cf. détail plus loin)
    * on obtient $s_{t+1},r_t=env(s_t,a_t)$ et on stocke $(s_t,a_t,r_t,s_{t+1})$ dans le buffer `R`
    * on tire un batch de $(s_i,a_i,r_i,s_{i+1})$
    * on définit  $y_i= r_i + \gamma  Q(s_{i+1}, \mu(s_{i+1})) $.
    * Entrainement du critique: on change les poids $w_Q$ de $Q$ pour minimiser la distance entre les $y_i$ et les $Q(s_i,a_i)$ (car dans l'idéal ils sont égaux)
    * Entrainement de l'acteur : on change les poids $w_\mu$ de $\mu$ pour  maximiser $Q(s_i,\mu(s_i))$ (car dans il doit se rapprocher de la Q-fonction optimale).
    
    
* `end for`


`end for`

### Version double

Tout comme pour le DQN, on va dupliquer les réseaux de neurones pour lisser le processus. Les réseaux "target" veront leurs poids évoluer plus lentement.

On choisis des  réseaux de neurones:
* Le critique $Q(s,a)$
* Le critique-target $\tilde Q(s,a)$
* L'acteur $\mu(s)$
* L'acteur-target $\tilde \mu(s)$

Au départ les poids de $Q$ et de $\tilde Q$ sont égaux, ainsi que ceux $\mu$ et $\tilde \mu$

On initialise le buffer `R` qui a une capacité limité (first-in first-out). Ainsi les anciens enregistrement (moins bon que les récent) seront oubliés.

`for` episode = 1,M `do`
* On choisit un état initial $s_1$
* `for` $t=1,...,T$ `do`
    * on sélectionne une action $a_t$ selon une politique d'exploration qui est basée sur $\mu$ (cf. détail plus loin)
    * on obtient $s_{t+1},r_t=env(s_t,a_t)$ et on stocke $(s_t,a_t,r_t,s_{t+1})$ dans le buffer `R`
    * on tire un batch de $(s_i,a_i,r_i,s_{i+1})$
    * on définit  $y_i= r_i + \gamma \tilde Q(s_{i+1},\tilde \mu(s_{i+1})) $. C'est uniquement à cette endroit qu'on utilise les targets modèles (qui évoluent plus lentement).  
    * Entrainement du critique: on change les poids $w_Q$ de $Q$ pour minimiser la distance entre les $y_i$ et les $Q(s_i,a_i)$ (car dans l'idéal ils sont égaux)
    * Entrainement de l'acteur : on change les poids $w_\mu$ de $\mu$ pour  maximiser $Q(s_i,\mu(s_i))$ (car dans il doit se rapprocher de la Q-fonction optimale).
    *  Update les modèles targets: On se fixe une constante $\tau$ (ex: $0.1$) et on modifie légèrement les poids des 2 modèles targets:
    $$
    w_{\tilde Q} \leftarrow (1-\tau) w_{\tilde Q} + \tau w_ Q
    $$
    $$
    w_{\tilde \mu} \leftarrow (1-\tau) w_{\tilde \mu} + \tau w_\mu
    $$
    
* `end for`


`end for`





### Les politiques d'exploration


* Supposons que l'ensemble des actions est continue: que c'est une partie de $\mathbb R^n$. Dans ce cas on prend pour $\mu$ un réseau de neurone regresseur à valeur dans $\mathbb R^n$. Le choix d'une exploration sera alors donné par
$$
a_t= \mu(s_t) + Bruit_t
$$
On choisit en général un processus de Bruit stationnaire, Gaussien et "continue": un processus Ornstein Uhlenbeck. L'agorithme dans ce cadre là s'appelle le DDPG (Deep Deterministic Policy Gradient).


* Supposons que l'ensemble des actions des discret: $\{1,2,3,...,k\}$. On prend pour $\mu$ un classifier à $k$ classes. Et le choix d'une exploration est alors donnée par une action $a_t$ tirée selon les probas données par $\mu(s_t)$.  


* Dans les deux cas, on peut aussi prendre comme action une prédiction de l'acteur $\mu$ mais en bruitant légèrement ses poids. C'est l'idée développée dans cette article [PARAMETER SPACE NOISE FOR EXPLORATION](https://arxiv.org/pdf/1706.01905.pdf)



## import

In [ ]:
%reset -f

In [ ]:
DO_TEST=True #mettre a False pour faire tourner le notebook plus vite

In [ ]:
import gym
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from tensorflow import keras
#import keras

from datetime import datetime
import pickle
import json
import time
import pytz #pour l'heure locale

from IPython.display import clear_output

In [ ]:
import os
CURENT_DIR="test_ddpf_in_the_box"

if not os.path.exists(CURENT_DIR):
    os.makedirs(CURENT_DIR)
else:
    print("outdir already created, its content is:")
    print(os.listdir(CURENT_DIR))

## Environement

### Utils

In [ ]:
class Space:
    def __init__(self,shape,low,high):
        assert len(low) == shape[0] == len(high)
        self.shape=shape #ex [2]
        self.low=low #ex [-2,-1]
        self.high=high #ex [3,2]

    def sample(self):
        return np.random.uniform(self.low,self.high,size=self.shape[0])

In [ ]:
class Show_trajectory:

    def __init__(self,one_dim,env):
        self.one_dim=one_dim
        self.env=env
        self.reset()

    def update(self,old_state):
        if self.one_dim:
            self.x.append(self.time)
            self.y.append(old_state[0])
        else:
            self.x.append(old_state[0])
            self.y.append(old_state[1])

        self.time+=1

    def reset(self):
        self.time=0
        self.x=[]
        self.y=[]

    def show(self):
        fig,ax=plt.subplots()

        if self.one_dim:
            ax.set_xlabel("time")
            ax.set_ylabel("space")
            ax.set_ylim([self.env.observation_space.low[0],self.env.observation_space.high[0]])
        else:
            ax.set_xlabel("first coordinate")
            ax.set_ylabel("second coordinate")
            ax.set_xlim([self.env.observation_space.low[0],self.env.observation_space.high[0]])
            ax.set_ylim([self.env.observation_space.low[1],self.env.observation_space.high[1]])

        ax.plot(self.x[0],self.y[0],"o")
        ax.plot(self.x,self.y,".-")
        plt.show()

### La Bille

In [ ]:
class Bille_Env:

    def __init__(self,dimension,sigma=1):

        self.dimension=dimension
        self.sigma=sigma

        self.name="Bille"

        low=[-10]*dimension
        high=[+10]*dimension
        shape=[dimension]
        #API gym
        self.observation_space=Space(shape,low,high)

        low=[-2]*dimension
        high=[+2]*dimension
        #API gym
        self.action_space=Space(shape,low,high)

        #API gym
        self._max_episode_steps=500

        self._do_render=False
        self.monitor = Show_trajectory(self.dimension==1,self)

        self.reset()

    #API gym
    def reset(self):
        self.position=np.random.uniform(self.action_space.low,self.action_space.high,size=self.dimension)
        self.count=0
        return self.position

    #API gym
    def step(self,action):

        self.count+=1

        assert len(action)==self.dimension, "bad dimension for action"
        for i in range(self.dimension):
            assert self.action_space.low[i]<=action[i]<=self.action_space.high[i], "action out of range"


        next_position=self.position + action + self.sigma*np.random.normal(size=self.dimension)

        terminal_bad=False
        terminal_good=False

        # on pert si la bille sort d'un rectangle
        for i in range(self.dimension):
            inside= self.observation_space.low[i]<=self.position[i]<=self.observation_space.high[i]
            if not inside:
                terminal_bad=True
                break

        if terminal_bad: reward=-10
        else : reward= 1

        #on gagne si la bille reste 500 fois
        if self.count>500:
            terminal_good=True
            reward=50

        self.position=next_position
        self.monitor.update(next_position)

        terminal=terminal_bad or terminal_good
        if terminal:
            self.reset() #la position est réinitialiser
            if (self._do_render):
                self.monitor.show()
                self.monitor.reset()

        return next_position,reward,terminal,{}

    #API gym
    def render(self):
        self._do_render=True

    def stop_render(self):
        self._do_render=False

    #API gym
    def close(self):
        pass

### Test Env

In [ ]:
def test_env(env,do_render=False):

    num_states = env.observation_space.shape[0]
    print("dim of State Space ->  {}".format(num_states))
    num_actions = env.action_space.shape[0]
    print("dim of Action Space ->  {}".format(num_actions))

    upper_bound = env.action_space.high
    lower_bound = env.action_space.low

    print("Max Value of Action ->  {}".format(upper_bound))
    print("Min Value of Action ->  {}".format(lower_bound))

    print("un état",env.reset())
    print("une action",env.action_space.sample())
    print("step: \n next_state,reward,done,info \n",env.step(env.action_space.sample()))

    if do_render:
        env.render()
    for i in range(100):
        next_state,reward,done,info=env.step(env.action_space.sample())


In [ ]:
if DO_TEST:
    test_env(Bille_Env(2),True)

## Explorateurs


In [ ]:
#todo: passer à la dim n
class OU_noise_generator:

    def __init__(self, dim, std=0.2, theta=0.15, dt=1e-2):
        self.dim=dim
        self.theta = theta
        self.std = std
        self.dt = dt
        self.reset()

    def generate(self):
        #     X_t+1 =  X_t  -  theta X_t * dt   + sigma * sqrt(dt) * N(0,1)
        self.x_prev = self.x_prev  - self.theta *  self.x_prev * self.dt + self.std * np.sqrt(self.dt) * np.random.normal(size=self.dim)
        return self.x_prev

    def drift_me(self,drift):
        self.x_prev+=drift

    def reset(self):
        self.x_prev=np.zeros([self.dim])


In [ ]:
def test_OU_noise_generator():

    dim=10
    t1=500
    t2=500
    res=np.zeros([t1+t2,dim])

    generator=OU_noise_generator(dim)

    for i in range(t1):
        res[i,:]=generator.generate()
        generator.drift_me(0.5*generator.dt)

    generator.reset()

    for i in range(t2):
        res[t1+i,:]=generator.generate()

    for j in range(dim):
        plt.plot(res[:,j])


if DO_TEST:
    test_OU_noise_generator()

In [ ]:
def smile(x,a,b):
    return ((x-a)/(b-a)*np.pi-np.pi/2)**6 #ne pas mettre de fonction tangente

def smile_derivative(x,a,b):
    return ((x-a)/(b-a)*np.pi-np.pi/2)**5 * 6 /(b-a)*np.pi

def test_smile():
    a=np.array([-1,-1])
    b=np.array([1,2])
    x=np.linspace(a+0.1,b-0.1,1000)

    fig,(ax0,ax1)=plt.subplots(2,1,sharex=True,figsize=(8,10))
    y=smile(x,a,b)
    y_prime=smile_derivative(x,a,b)
    print("x.shape",x.shape)
    print("y.shape",y.shape)
    print("y_prime.shape",y_prime.shape)
    ax0.plot(x[:,0],y[:,0],label="smile")
    ax0.plot(x[:,0],y_prime[:,0],label="derivative")
    ax1.plot(x[:,1],y[:,1],label="smile")
    ax1.plot(x[:,1],y_prime[:,1],label="derivative")
    ax0.legend()
    ax1.legend()

if DO_TEST:
    test_smile()

In [ ]:
class OU_Explorator:

    # state_to_action_fn: prend un vecteur de taille dim-state et renvoie un vecteur de taille dim-action
    def __init__(self,dim_action,lower_bounds,upper_bounds,repulsive_boundary):
        assert len(lower_bounds)==len(upper_bounds)==dim_action
        self.dim_action=dim_action
        self.noise_object = OU_noise_generator(self.dim_action)
        self.repulsive_boundary=repulsive_boundary
        self.lower_bounds=lower_bounds
        self.upper_bounds=upper_bounds

    def explore(self,pre_action):
        assert len(pre_action)==self.dim_action

        action = pre_action + self.noise_object.generate()

        # We make sure action is within bounds
        if self.repulsive_boundary:
            grad=smile_derivative(action, self.lower_bounds, self.upper_bounds)
            action-=grad*1e-4
            self.noise_object.drift_me(-grad*1e-4)

        action = np.clip(action, self.lower_bounds, self.upper_bounds)
        return action

    def reset(self):
        self.noise_object.reset()

Quand repulsive_boundary=False: phénomène de saturation: Quand le processus de OU fait une grande excursion,  l'action peut rester cliper assez longtemps.



In [ ]:
def test_OU_Explorator(repulsive_boundary):
    dim=5
    explorer=OU_Explorator(dim,np.array([0]*dim),np.array([1]*dim),repulsive_boundary)

    tmax=700
    res=np.zeros([tmax,dim])
    state=np.ones([3])

    for i in range(tmax):
        res[i,:]=explorer.explore(np.ones([dim])*0.8)

    for j in range (dim):
        plt.plot(res[:,j],label=str(j))



In [ ]:
if DO_TEST:
    test_OU_Explorator(False)

In [ ]:
if DO_TEST:
    test_OU_Explorator(True)

## Modèles




Il faudrait étudier la question de la batchNormalisation (qui pourrait faire baisser la variance du aux simu très disparates).

### update_target

In [ ]:
"""
La moyennation des poids a un coup non negligeable => on utilise @tf.function
"""
@tf.function
def update_target(target_model, model,tau):
    target_weights=target_model.trainable_variables
    weights=model.trainable_variables
    for (a, b) in zip(target_weights, weights):
        a.assign(b * tau + a * (1 - tau))

### Critic

Un réseau de neurone $Q[s,a]$ qui sort un scalaire. Il doit in fine s'approcher de $Q^{op}$.

            {
            "action_layer_dims":(16,32),
            "state_layer_dims": (17,33),
            "common_layer_dims":(60,70),
            "critic_layer_dims":(12,),
            "actor_layer_dims": (24,)
            }


On obtient pour le critic
    
    action -16-32-
                   \
                    \
                     -60-70- 12-1
                    /
    state -17-33-  /   


Et pour l'acteur, dans le cas où la dimension des action est de 4:
    
                     -60-70- 24-4
                    /
    state -17-33-  /   






In [ ]:
default_model_struct={
                    "action_layer_dims":(16,32),
                    "state_layer_dims":(16,32),
                    "common_layer_dims":(64,64),
                    "critic_layer_dims":(64,),
                    "actor_layer_dims":(64,)
                    }

In [ ]:
class Critic(tf.keras.Model):

    def __init__(self,dim_state,dim_action,model_struct=default_model_struct):

        self.state_layers=[]
        for i in model_struct["state_layer_dims"]:
            self.state_layers.append(layers.Dense(i, activation="relu"))

        self.action_layers=[]
        for i in model_struc["action_layer_dims"]:
            self.action_layers.append(layers.Dense(i, activation="relu"))

        self.common_layers=[]
        for i in model_struc["common_layer_dims"]:
            self.common_layers.append(layers.Dense(i, activation="relu"))

        self.final_layer=layers.Dense(1)




    def call(self, s,a):
        for lay in self.state_layers:
            s=lay(s)

        for lay in self.action_layers:
            a=lay(a)

        sa=tf.concat([s,a],axis=1)
        for lay in self.common_layers:
            sa=lay(sa)

        return self.final_layer(sa)





In [ ]:
class ActorCritic(tf.keras.Model):

    def __init__(self,dim_state,dim_action,low_action,up_action,model_struct=default_model_struct):


        self.low_action=tf.constant(low_action) #ex: (-1,-2)
        self.up_action=tf.constant(up_action)  #ex:(1,2)
        assert len(self.up_action.shape)==dim_action
        assert len(self.down_action.shape)==dim_action


        self.state_layers=[]
        for i in model_struct["state_layer_dims"]:
            self.state_layers.append(layers.Dense(i, activation="relu"))

        self.action_layers=[]
        for i in model_struc["action_layer_dims"]:
            self.action_layers.append(layers.Dense(i, activation="relu"))

        self.common_layers=[]
        for i in model_struc["common_layer_dims"]:
            self.common_layers.append(layers.Dense(i, activation="relu"))

        self.final_layer=layers.Dense(1)

        self.final_layer_action=layers.Dense(dim_action,activation="sigmoid")



    def call(self, s,a):
        for lay in self.state_layers:
            s=lay(s)

        for lay in self.action_layers:
            a=lay(a)

        sa=tf.concat([s,a],axis=1)
        for lay in self.common_layers:
            sa=lay(sa)

        a_=self.final_layer_action(s)
        a_ = a_*(self.up_action-self.down_action)[None,:]+self.down_action[None,:]


        q=self.final_layer(sa)

        return a_,q





In [ ]:
def make_critic(
                dim_state,
                dim_action,
                model_struct=default_model_struct
                ):

    state_input = keras.layers.Input([dim_state])
    action_input=keras.layers.Input([dim_action])

    state_out=   state_input
    # state as input
    for i in model_struct["state_layer_dims"]:
        state_out=layers.Dense(i, activation="relu")(state_out)

    # Action as input
    action_out=action_input
    for i in model_struct["action_layer_dims"]:
        action_out=layers.Dense(i, activation="relu")(action_out)


    concat = layers.Concatenate()([state_out, action_out])
    for i in model_struct["common_layer_dims"]:
        concat=layers.Dense(i,activation="relu")(concat)

    critic=concat
    for i in model_struct["critic_layer_dims"]:
        critic=layers.Dense(i, activation="relu")(critic)

    critic=layers.Dense(1)(critic)

    model=keras.Model(inputs = [state_input,action_input], outputs = critic)

    return model


def test_critic():
    model=make_critic(2,1)

    model.summary()

    state=tf.ones([50,2])
    action=tf.ones([50,1])

    print("result:",model([state,action]).shape)

In [ ]:
if DO_TEST:
    test_critic()

### Actor



In [ ]:
def make_actor(
                dim_state,
                dim_action,
                lower_bounds,
                upper_bounds,
                model_struct=default_model_struct
                ):

    lower_bounds=tf.constant(lower_bounds,dtype=tf.float32)
    upper_bounds=tf.constant(upper_bounds,dtype=tf.float32)
    assert lower_bounds.shape==upper_bounds.shape==(dim_action,),"bad bounds"

    # state as input
    input_state=keras.Input([dim_state])

    input_out=input_state
    for i in model_struct["state_layer_dims"]:
        input_out=layers.Dense(i, activation="relu")(input_out)

    concat=input_out
    for i in model_struct["common_layer_dims"]:
        concat=layers.Dense(i,activation="relu")(concat)

    actor=concat
    for i in model_struct["actor_layer_dims"]:
        actor=layers.Dense(i,activation="relu")(actor)

    actor=layers.Dense(dim_action,activation="sigmoid")(actor)

    low=lower_bounds[tf.newaxis,:]
    up=upper_bounds[tf.newaxis,:]
    actor=actor*(up-low)+low

    model=keras.Model(inputs = input_state, outputs = actor)

    return model

In [ ]:
def test_actor():
    model=make_actor(2,3,[0,0,0],[10,10,10])

    model.summary()
    state=np.random.uniform(-5,5,size=[50,2])
    res=model(state)
    print("result:",res.shape)
    print("min,max:",np.min(res),np.max(res))

if DO_TEST:
    test_actor()

## Agent and co

Attention, certain paramètre sont très sensible. Notamment $\tau$.

### History

In [ ]:
class History:
    def __init__(self):
        self.episodes=[]
        self.episodes_duration=[]
        self.episodes_nb_step =[]
        self.total_rewards= []
        self.recent_total_rewards_averages = []
        self.current_episode=0


    def update(self,i,total_reward,episode_duration):
        self.current_episode+=1
        self.episodes.append(self.current_episode)
        self.episodes_nb_step.append(i)
        self.total_rewards.append(total_reward)
        self.recent_total_rewards_averages.append(sum(self.total_rewards[-50:]) / len(self.total_rewards[-50:]) )
        self.episodes_duration.append(episode_duration)

    def show(self,to_log):
        print(f"episode: {self.current_episode}, duration:{self.episodes_duration[-1]:.2f} , total reward:{self.total_rewards[-1]}, rec-tot-rew-av: {self.recent_total_rewards_averages[-1]:.2f}"+to_log)


    def plot(self):
        fig,ax=plt.subplots()
        time_chrono=np.cumsum(self.episodes_duration)
        ax.plot(time_chrono, self.total_rewards, 'b')
        ax.plot(time_chrono, self.recent_total_rewards_averages, 'r')
        ax.set_ylabel('total reward', fontsize=18)
        ax.set_xlabel('duration', fontsize=18)
        plt.show()


### Agent

In [ ]:
class Agent:

    def __init__(self,
                 env,
                 target_update=(0.005,1), #(tau,interval)
                 buffer_capacity=5000,#50000
                 batch_size=64,
                 gamma = 0.99, # Discount factor
                 lr=1e-3, #learning rate
                 repulsive_boundary=True,
                 model_struct=default_model_struct
                 ):

        self.env=env
        self.target_update=target_update
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.gamma = gamma
        self.lr=lr
        self.repulsive_boundary=repulsive_boundary
        self.model_struct=model_struct

        self.dim_state = self.env.observation_space.shape[0]
        self.dim_action = self.env.action_space.shape[0]

        #bounds for actions
        self.lower_bounds=np.array(self.env.action_space.low)
        self.upper_bounds=np.array(self.env.action_space.high)


        self.verbose=0
        self.history_train=History()
        self.history_test=History()


        self.initialize_models()

        # Its tells us num of times record() was called.
        self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, self.dim_state))
        self.action_buffer = np.zeros((self.buffer_capacity, self.dim_action))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, self.dim_state))

        self.explorer=OU_Explorator(self.dim_action,self.lower_bounds,self.upper_bounds,repulsive_boundary=self.repulsive_boundary)

        self.global_ite_count=0


    def initialize_models(self):

        self.actor = make_actor(self.dim_state,self.dim_action,self.lower_bounds,self.upper_bounds,self.model_struct)
        self.critic= make_critic(self.dim_state,self.dim_action,self.model_struct)

        self.target_actor = make_actor(self.dim_state,self.dim_action,self.lower_bounds,self.upper_bounds)
        self.target_critic= make_critic(self.dim_state,self.dim_action)
        #transfert des poids
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())

        self.critic_optimizer = tf.keras.optimizers.Adam(self.lr)
        self.actor_optimizer =  tf.keras.optimizers.Adam(self.lr)


    def save_weights(self):
        #on n'enregistre pas les targets_model
        self.actor_weights=self.actor.get_weights()
        self.critic_weights=self.critic.get_weights()

    def recover_good_weights(self):
        self.actor.set_weights(self.actor_weights)
        self.target_actor.set_weights(self.actor_weights)
        self.critic.set_weights(self.critic_weights)
        self.target_critic.set_weights(self.critic_weights)


    #  (s,a,r,s') = (state,action,reward,next_state)
    def record(self, s,a,r,s_):
        # le modulo permet de remplacer les anciens enregistremenets
        index = self.buffer_counter % self.buffer_capacity
        self.state_buffer[index] = s
        self.action_buffer[index] = a
        self.reward_buffer[index] = r
        self.next_state_buffer[index] = s_
        self.buffer_counter += 1


    @tf.function
    def update(self, state, action, reward, next_state):

        # Entrainement du critique
        with tf.GradientTape() as tape:
            target_action = self.target_actor(next_state)
            # on veut que le critique vérifie de plus en plus bellman
            y = reward + self.gamma * self.target_critic([next_state,target_action ])
            critic_value = self.critic([state, action])
            # pour que l'on satisfasse de plus en plus bellmann
            critic_loss = tf.math.reduce_mean((y - critic_value)**2)

        critic_grad = tape.gradient(critic_loss, self.critic.trainable_variables)
        self.critic_optimizer.apply_gradients(zip(critic_grad, self.critic.trainable_variables))

        #Entrainement de l'acteur
        with tf.GradientTape() as tape:
            critic_value = self.critic([state, self.actor(state)])
            # L'acteur veut maximiser la valeur de son action donnée par le critique.
            # Pour maximiser on met un signe -
            actor_loss = -tf.math.reduce_mean(critic_value)

        actor_grad = tape.gradient(actor_loss, self.actor.trainable_variables)
        self.actor_optimizer.apply_gradients(zip(actor_grad, self.actor.trainable_variables))


    # We compute the loss and update parameters
    def learn(self):
        # Get sampling range
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
        state = tf.constant(self.state_buffer[batch_indices],dtype=tf.float32)
        action = tf.constant(self.action_buffer[batch_indices],dtype=tf.float32)
        reward = tf.constant(self.reward_buffer[batch_indices],dtype=tf.float32)
        next_state = tf.constant(self.next_state_buffer[batch_indices],dtype=tf.float32)

        self.update(state, action, reward, next_state)



    def policy(self,state):
        state=tf.constant(state,dtype=tf.float32)[tf.newaxis,:]
        sampled_action = self.actor(state)[0]
        return self.explorer.explore(sampled_action)


    def _run(self,minutes,testing:bool):
        #testing=True => test(), sinon train()

        init_time=time.time()
        episodes_count=0

        try:
            OK=True
            while OK:
                episodes_count+=1
                OK=time.time()-init_time<minutes*60

                prev_state = self.env.reset()
                self.explorer.reset()
                episodic_reward = 0

                self.initial_time=time.time()
                current_step=0


                #attention, l'environnnement ne doit pas renvoyer d'épisode de longueur infini
                #amélioration (pas forcement)  interompre le train dès que le temps est dépassé, même si l'épisode n'est pas fini
                done=False
                while not done:#1 épisode
                    self.global_ite_count+=1

                    action = self.policy(prev_state)
                    state, reward, done, info = self.env.step(action)
                    episodic_reward += reward

                    if not testing:
                        self.record(prev_state, action, reward, state)
                        self.learn()
                        #target_update est une paire (tau,update_interval)
                        if  self.target_update is not None and self.global_ite_count%self.target_update[1]==0:
                            update_target(self.target_actor,self.actor,self.target_update[0])
                            update_target(self.target_critic,self.critic,self.target_update[0])

                    prev_state = state
                #FIN DE L'ÉPISODE


                #enregistrement
                episode_duration=time.time()-self.initial_time
                if not testing:
                    self.history_train.update(current_step,episodic_reward,episode_duration)
                    #a chaque record, on sauve les poids du réseau de neurone
                    # on peut considérer les records smooth ou pas (a réfléchir selon les cas)
                    #if len(self.history_train.recent_total_rewards_averages)>20 and self.history_train.recent_total_rewards_averages[-1]>=max(self.history_train.recent_total_rewards_averages[20:]):
                    if self.history_train.total_rewards[-1]>=max(self.history_train.total_rewards):
                        print(f"↗{self.history_train.total_rewards[-1]:.1f}", end="")
                        self.save_weights()
                        print('.', end='')
                else:
                    self.history_test.update(current_step,episodic_reward,episode_duration)
                    print(f"↗{self.history_test.total_rewards[-1]:.1f}", end="")





        except KeyboardInterrupt :
            pass
        self.env.close()


    def train(self,minutes):
        self._run(minutes,testing=False)

    def test(self,minutes):
        self._run(minutes,testing=True)


## Entrainement



L'agent s'améliore jusqu'à arriver à la performance max. Et ensuite il craque.

In [ ]:
agent=Agent(Bille_Env(2,2),lr=1e-3)

In [ ]:
agent.train(1)
agent.history_train.plot()

In [ ]:
agent.recover_good_weights()
agent.test(0.5)
agent.history_test.plot()

## A vous: Pendulum

### Description

    
The inverted pendulum swingup problem is based on the classic problem in control theory.

The system consists of a pendulum attached at one end to a fixed point, and the other end being free.

The pendulum starts in a random position and the goal is to apply torque on the free end to swing it into an upright position, with its center of gravity right above the fixed point.


    
The diagram below specifies the coordinate system used for the implementation of the pendulum's dynamic equations.

-  `x-y`: cartesian coordinates of the pendulum's end in meters.
- `theta` : angle in radians.
- `tau`: torque in `N m`. Defined as positive _counter-clockwise_.


### Action Space

The action is a `ndarray` with shape `(1,)` representing the torque applied to free end of the pendulum.

| Num | Action | Min  | Max |
|-----|--------|------|-----|
| 0   | Torque | -2.0 | 2.0 |

### Observation Space

The observation is a `ndarray` with shape `(3,)` representing the x-y coordinates of the pendulum's free end and its angular velocity.

| Num | Observation      | Min  | Max |
|-----|------------------|------|-----|
| 0   | x = cos(theta)   | -1.0 | 1.0 |
| 1   | y = sin(theta)   | -1.0 | 1.0 |
| 2   | Angular Velocity | -8.0 | 8.0 |

### Rewards

The reward function is defined as:

*r = -(theta<sup>2</sup> + 0.1 * theta_dt<sup>2</sup> + 0.001 * torque<sup>2</sup>)*


where `$\theta$` is the pendulum's angle normalized between *[-pi, pi]* (with 0 being in the upright position).

Based on the above equation, the minimum reward that can be obtained is

*-(pi<sup>2</sup> + 0.1 * 8<sup>2</sup> + 0.001 * 2<sup>2</sup>) = -16.2736044*

while the maximum reward is zero (pendulum is upright with zero velocity and no torque applied).

### Starting State

The starting state is a random angle in [-pi, pi] and a random angular velocity in [-1,1].

### Episode Truncation

The episode truncates at 200 time steps.

### Arguments

- `g`: acceleration of gravity measured in *(m s<sup>-2</sup>)* used to calculate the pendulum dynamics. The default value is g = 10. You can change it like this to be more realistic:
    ```
    gym.make('Pendulum-v2', g=9.81)
    ```


### Quelques test

Expliquez ce qu'on fait dans les programmes suivants

In [ ]:
if DO_TEST:
    test_env(gym.make("Pendulum-v1"),do_render=False)

In [ ]:
env=gym.make('Pendulum-v1')

print("initialisation:",env.reset())

ss=[]
rs=[]
done=False
while not done:
    action=[0.1]
    s,r,done,info=env.step(action)
    ss.append(s)
    rs.append(r)
env.close()

In [ ]:
ss=np.stack(ss)
ss.shape

In [ ]:
import numpy as np
plt.plot(ss[:,0])

In [ ]:
plt.plot(rs);

In [ ]:
done=False
while not done:
    s,r,done,info=env.step([0.1])
print("s,r,done",s,r,done)

In [ ]:
for i in range(10):
    s,r,done,info=env.step([0.1])
    print(s,r,done)

Mainenant entrainer l'agent pour qu'il puisse maintenir le pendule en l'air un maximum de temps.